In [0]:
from google.colab import drive
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)

In [0]:
df['brand_cat'] = df.brand.factorize()[0]
feats = ['brand_cat']

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring= 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [16]:
run_model(feats)

(-58.38655694633361, 4.223555478221712)

In [17]:
run_model(feats, RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0))

(-57.47223572384038, 4.328288468270897)

# Feature Engineering

In [11]:
df['brand_cat_lower'] = df.brand.map(lambda x: str(x).lower()).factorize()[0]
feats = ['brand_cat_lower']
run_model(feats, RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0))

(-57.31783843165656, 4.181246596160967)

In [12]:
# Colum features has a structure of dictionary
df.features.head(1).values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'],
      dtype=object)

In [0]:
# function to parse features column
def parse_features(x):
  if str(x) == 'nan' : return []
  return literal_eval(x.replace('\\"', '"')) # replacing unexpected pattern that raise error

df['features_parsed'] = df['features'].map(parse_features)

In [14]:
df.features_parsed.head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [0]:
# better function to parse features column
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict
  features = literal_eval(x.replace('\\"', '"')) # replacing unexpected pattern that raise error

  #process created clean dictionary
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict
df['features_parsed'] = df['features'].map(parse_features)

In [16]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [17]:
# how many unique values?

keys = set()

df.features_parsed.map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [19]:
# create columns from dictionary keys

# function for naming new columns
def get_feat_name(key):
  return "feat_" + key

for key in tqdm_notebook(keys):
  df[get_feat_name(key)] = df.features_parsed.map(lambda feats: feats[key] if str(key) in feats else np.nan)


In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_feat_name(key)].isnull() ].shape[0] / df.shape[0] * 100

keys_stat

In [23]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df.feat_brand.factorize()[0]
df['feat_color_cat'] = df.feat_color.factorize()[0]
df['feat_gender_cat'] = df.feat_gender.factorize()[0]
df['feat_manufacturer_part_number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df.feat_material.factorize()[0]

In [28]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat_lower'], model)

(-57.31783843165656, 4.181246596160967)

In [29]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat_lower','feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', \
           'feat_manufacturer_part_number_cat', 'feat_material_cat'], model)

(-57.172328503725836, 4.227341275008907)

In [41]:
feats = ['brand_cat_lower','feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', \
           'feat_manufacturer_part_number_cat', 'feat_material_cat']
X = df[ feats ].values
y = df[ 'prices_amountmin'].values
model.fit(X, y)
result = run_model(['brand_cat_lower','feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', \
           'feat_material_cat'], model)
print(result)
perm = PermutationImportance(model, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names = feats)

(-57.12896870527643, 4.1608099520676705)


Weight,Feature
0.2565 ± 0.0110,brand_cat_lower
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer_part_number_cat


In [51]:
!git push

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 5.69 KiB | 1.90 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/michkam89/dw-matrix.git
   7b6a003..7e9c521  master -> master
